In [31]:
%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [32]:
%pig_start

In [33]:
%timeout 300

In [34]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
Deleted output/_SUCCESS
Deleted output/part-r-00000
rm: `output2/*': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [35]:
!hadoop fs -put *.csv .
!hadoop fs -ls

put: `data.csv': File exists
Found 2 items
-rw-r--r--   1 root supergroup        615 2020-01-19 13:26 data.csv
-rw-r--r--   1 root supergroup       1741 2020-01-19 15:51 data.tsv


In [36]:
!rm -rf input
!rm -rf output
!mkdir input
!mkdir output

In [37]:
%%writefile script.pig

-- crea la carpeta input in el HDFS
-- fs -mkdir input

-- copia los archivos del sistema local al HDFS
fs -put input/ .

-- carga de datos
--Punto 10

u = LOAD 'data.csv' USING PigStorage(',')
    AS (f1:INT, f2:CHARARRAY, f3:CHARARRAY, f4:DATETIME, f5:CHARARRAY, f6:INT);
    
y = GROUP u BY $2;

z = FOREACH y GENERATE group, SIZE($0) as ld1:CHARARRAY;
    
zz = FOREACH z GENERATE CONCAT(group, ',' , ld1) as final:CHARARRAY;

zzz= FOREACH zz GENERATE final, SUBSTRING(final,1,-1) as 

y = ORDER zz BY SUBSTRING(final,1,-2) DESC;

w = STREAM y THROUGH  `head -n 5 `;

DUMP w;

-- escribe el archivo de salida
STORE w INTO 'output';

-- copia los warchivos del HDFS al sistema local
fs -get output/ .

Overwriting script.pig


In [38]:
!pig -execute 'run script.pig'

2020-01-19 17:29:56,265 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-01-19 17:30:00,103 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-01-19 17:30:00,511 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2020-01-19 17:30:00,519 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2020-01-19 17:30:00,536 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
2020-01-19 17:30:00,558 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.submit.replication is deprecated. Instead, use mapreduce.client.submit.file.

In [39]:
!hadoop fs -ls output/*

-rw-r--r--   1 root supergroup          0 2020-01-19 17:33 output/_SUCCESS
-rw-r--r--   1 root supergroup          5 2020-01-19 17:33 output/part-r-00000


In [40]:
!hadoop fs -cat output/part-r-00000

In [13]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
Deleted output/_SUCCESS
Deleted output/part-r-00000
rm: `output2/*': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [14]:
!rm -rf input
!rm -rf output